## Injection benchmark

### Setup

In [1]:
import sys
from antidote import __version__, is_compiled
print(f"Antidote: {__version__()} {'(cython)' if is_compiled() else ''}")
print(f"Python {sys.version}")

Antidote: 0.7.4.dev4+g5ddfb12.d20200822 
Python 3.8.5 (default, Aug 21 2020, 11:31:30) 
[GCC 9.3.0]


### Results
The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the most common use case of dependency injection, service instantiation, the overhead should be negligible.

It should be noted that in most cases the worst scenario is used, as functions do nothing. In the real world, pure python functions are a lot slower. So to put the following results into perspective, here is the time needed to decode this simple JSON.

In [2]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

2.64 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
from antidote import world, register, inject

@register
class Service1:
    pass


@register
class Service2:
    def __init__(self, service1: Service1):
        self.service1 = service1


@register
class Service3:
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@register
class Service4:
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

### Function call

Injection overhead is here measured with a function which does nothing.

In [4]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

Time necessary to only execute the function, without retrieving the services

In [5]:
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
%timeit f(*args)

129 ns ± 2.73 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Overhead of the injection when all argument must be retrieved from the container.

In [6]:
f_injected = inject(f)
assert f(*args) == f_injected()
%timeit f_injected()

3.97 µs ± 32.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [7]:
assert f(*args) == f_injected(*args)
%timeit f_injected(*args)

2.13 µs ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Non singleton

In [8]:
@register(singleton=False)
class ServiceX:
    pass

In [9]:
%timeit world.get(ServiceX)

6.65 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Custom Provider

In [10]:
from antidote import provider, world
from antidote.core import DependencyProvider, DependencyInstance
dep = object()
dep2 = object()
import time

@provider
class SlowProvider(DependencyProvider):
    def provide(self, dependency, container):
        if dep2 is dependency:
            return DependencyInstance("sleepy")
            

@provider
class CustomProvider(DependencyProvider):
    def provide(self, dependency, container):
        if dependency is dep:
            return DependencyInstance("yeah")


In [11]:
%timeit world.get(dep)

7.86 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Object instantiation

In [12]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

415 ns ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
@register
class ObjInjected:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

7.79 µs ± 392 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Configuration


In [14]:
from antidote import LazyConstantsMeta

class Conf(metaclass=LazyConstantsMeta):
    A = 'A'
    B = 'B'

    def get(self, key):
        return key

In [15]:
def g(a, b):
    return a, b

%timeit g('A', 'B')

102 ns ± 3.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [16]:
conf = Conf()
%timeit g(conf.get('A'), conf.get('B'))

303 ns ± 6.29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
assert (conf.A, conf.B) == (conf.get('A'), conf.get('B'))
%timeit g(conf.A, conf.B)

785 ns ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
g_injected = inject(g, dependencies=(Conf.A, Conf.B))

assert g(conf.get('A'), conf.get('B')) == g_injected()
assert g(conf.A, conf.B) == g_injected()

%timeit g_injected()

3.17 µs ± 38.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
